# A 2-Hours Seminar about [InterCom](https://github.com/Tecnologias-multimedia/intercom)

## 1. What is InterCom
InterCom is a real-time intercommunicator (currently, only transmitting audio). The sequence of [frames](https://python-sounddevice.readthedocs.io/en/0.3.12/api.html) (two stereo samples) is splitted into chunks and each one is transmitted in an [UDP](https://en.wikipedia.org/wiki/User_Datagram_Protocol) packet.

### A minimal implementation

In [1]:
# Interrupt with: Kernel -> Interrupt
!python ../minimal.py --show_stats

Unable to import argcomplete
argcomplete not working :-/
Running Minimal__verbose.__init__
Running Minimal.__init__
SECONDS_PER_CYCLE = 1
chunks_per_cycle = 43.06640625
frames_per_cycle = 44100

InterCom parameters:

Namespace(destination_address='localhost', destination_port=4444, frames_per_chunk=1024, frames_per_second=44100, input_device=None, list_devices=False, listening_port=4444, output_device=None, show_samples=False, show_stats=True)

Using device:

   0 HDA Intel PCH: ALC3246 Analog (hw:0,0), ALSA (2 in, 2 out)
   1 HDA Intel PCH: HDMI 0 (hw:0,3), ALSA (0 in, 8 out)
   2 HDA Intel PCH: HDMI 1 (hw:0,7), ALSA (0 in, 8 out)
   3 HDA Intel PCH: HDMI 2 (hw:0,8), ALSA (0 in, 8 out)
   4 HDA Intel PCH: HDMI 3 (hw:0,9), ALSA (0 in, 8 out)
   5 HDA Intel PCH: HDMI 4 (hw:0,10), ALSA (0 in, 8 out)
   6 sysdefault, ALSA (128 in, 128 out)
   7 front, ALSA (0 in, 2 out)
   8 surround40, ALSA (0 in, 2 out)
   9 surround51, ALSA (0 in, 2 out)
  10 surround71, ALSA (0 in, 2 out)
  11 hdmi, A

   28      43      43    1407    1407    3   14
Avgs:      43      43    1407    1407    3   16
cycle  mesgs.  mesgs.    kbps    kbps %CPU %CPU
         sent   recv.    sent   recv.    Global

   29      43      43    1406    1406    4   17
Avgs:      43      43    1407    1407    3   16
cycle  mesgs.  mesgs.    kbps    kbps %CPU %CPU
         sent   recv.    sent   recv.    Global

   30      44      44    1440    1440    4   13
Avgs:      43      43    1408    1408    3   16
cycle  mesgs.  mesgs.    kbps    kbps %CPU %CPU
         sent   recv.    sent   recv.    Global

   31      43      43    1406    1406    2   15
Avgs:      43      43    1408    1408    3   16
cycle  mesgs.  mesgs.    kbps    kbps %CPU %CPU
         sent   recv.    sent   recv.    Global

^C





CPU usage average = 3.7931371842251607 %
Payload sent average = 1408.6129032258063 kilo bits per second
Payload received average = 1408.6129032258063 kilo bits per second


## Theoretical bit-rate
$$
\frac{44100\frac{\text{frames}}{\text{second}}\times 2\frac{\text{samples}}{\text{frame}}\times 2\frac{\text{bytes}}{\text{sample}}\times 8\frac{\text{bits}}{\text{byte}}}{1000} = 1411.2 ~\text{kbps}
$$

## 2. Jitter and buffering
Non-dedicated links use to have a significant [jitter](https://en.wikipedia.org/wiki/Jitter).
To hide the it, we can use buffering:

![Buffering](https://tecnologias-multimedia.github.io/study_guide/buffering/graphics/timelines.svg)

### Which is the latency and jitter in my host?

In [3]:
!ping -c 10 localhost

PING localhost (127.0.0.1) 56(84) bytes of data.
64 bytes from localhost (127.0.0.1): icmp_seq=1 ttl=64 time=0.018 ms
64 bytes from localhost (127.0.0.1): icmp_seq=2 ttl=64 time=0.060 ms
64 bytes from localhost (127.0.0.1): icmp_seq=3 ttl=64 time=0.056 ms
64 bytes from localhost (127.0.0.1): icmp_seq=4 ttl=64 time=0.058 ms
64 bytes from localhost (127.0.0.1): icmp_seq=5 ttl=64 time=0.064 ms
64 bytes from localhost (127.0.0.1): icmp_seq=6 ttl=64 time=0.036 ms
64 bytes from localhost (127.0.0.1): icmp_seq=7 ttl=64 time=0.058 ms
64 bytes from localhost (127.0.0.1): icmp_seq=8 ttl=64 time=0.058 ms
64 bytes from localhost (127.0.0.1): icmp_seq=9 ttl=64 time=0.054 ms
64 bytes from localhost (127.0.0.1): icmp_seq=10 ttl=64 time=0.058 ms

--- localhost ping statistics ---
10 packets transmitted, 10 received, 0% packet loss, time 9205ms
rtt min/avg/max/mdev = 0.018/0.052/0.064/0.013 ms


Too good. Real links have much higher latencies.

### Let's increases latency and jitter for the `localhost` link
Let's try:
* Latency = 100 ms.
* Jitter = 100 ms.
* Correlation between RTTs = 0.25 (<1.0).
* Statistical distribution of the RTTs = normal.

In [5]:
# Remember to add "your_username_here ALL=(ALL) NOPASSWD: ALL" to the end of /etc/sudoers.
# Notice that these times express RTTs, not simple one-way lantencies.
!tc qdisc show dev lo
!sudo tc qdisc add dev lo root handle 1: netem delay 100ms 100ms 25% distribution normal
!tc qdisc show dev lo

qdisc noqueue 0: root refcnt 2 
qdisc netem 1: root refcnt 2 limit 1000 delay 100.0ms  100.0ms 25%


## Let's see ...

In [6]:
!ping -c 10 localhost

PING localhost (127.0.0.1) 56(84) bytes of data.
64 bytes from localhost (127.0.0.1): icmp_seq=1 ttl=64 time=180 ms
64 bytes from localhost (127.0.0.1): icmp_seq=2 ttl=64 time=208 ms
64 bytes from localhost (127.0.0.1): icmp_seq=3 ttl=64 time=331 ms
64 bytes from localhost (127.0.0.1): icmp_seq=4 ttl=64 time=200 ms
64 bytes from localhost (127.0.0.1): icmp_seq=5 ttl=64 time=213 ms
64 bytes from localhost (127.0.0.1): icmp_seq=6 ttl=64 time=405 ms
64 bytes from localhost (127.0.0.1): icmp_seq=7 ttl=64 time=102 ms
64 bytes from localhost (127.0.0.1): icmp_seq=8 ttl=64 time=234 ms
64 bytes from localhost (127.0.0.1): icmp_seq=9 ttl=64 time=284 ms
64 bytes from localhost (127.0.0.1): icmp_seq=10 ttl=64 time=117 ms

--- localhost ping statistics ---
10 packets transmitted, 10 received, 0% packet loss, time 9003ms
rtt min/avg/max/mdev = 102.113/227.520/404.770/87.751 ms


OK, as we can see, at least 100 ms of latency is added to the sent packets and also to the received packets. Total, approx: 200 ms.

### Let's listen again to minimal

In [7]:
!python ../minimal.py --show_stats

Unable to import argcomplete
argcomplete not working :-/
Running Minimal__verbose.__init__
Running Minimal.__init__
SECONDS_PER_CYCLE = 1
chunks_per_cycle = 43.06640625
frames_per_cycle = 44100

InterCom parameters:

Namespace(destination_address='localhost', destination_port=4444, frames_per_chunk=1024, frames_per_second=44100, input_device=None, list_devices=False, listening_port=4444, output_device=None, show_samples=False, show_stats=True)

Using device:

   0 HDA Intel PCH: ALC3246 Analog (hw:0,0), ALSA (2 in, 2 out)
   1 HDA Intel PCH: HDMI 0 (hw:0,3), ALSA (0 in, 8 out)
   2 HDA Intel PCH: HDMI 1 (hw:0,7), ALSA (0 in, 8 out)
   3 HDA Intel PCH: HDMI 2 (hw:0,8), ALSA (0 in, 8 out)
   4 HDA Intel PCH: HDMI 3 (hw:0,9), ALSA (0 in, 8 out)
   5 HDA Intel PCH: HDMI 4 (hw:0,10), ALSA (0 in, 8 out)
   6 sysdefault, ALSA (128 in, 128 out)
   7 front, ALSA (0 in, 2 out)
   8 surround40, ALSA (0 in, 2 out)
   9 surround51, ALSA (0 in, 2 out)
  10 surround71, ALSA (0 in, 2 out)
  11 hdmi, A

Quite bad :-/ The chunks don't arrive with a constant cadence to the receiver (see the previous figure).

### Let's listen to the version of InterCom that uses a buffer of 300 ms

In [8]:
!python ../buffer.py -b 300 --show_stats

Unable to import argcomplete
Unable to import argcomplete
argcomplete not working :-/
Running Buffering__verbose.__init__
Running Buffering.__init__
Running Minimal__verbose.__init__
Running Minimal.__init__
SECONDS_PER_CYCLE = 1
chunks_per_cycle = 43.06640625
frames_per_cycle = 44100
buffering_time = 300 miliseconds
chunks_to_buffer = 13

InterCom parameters:

Namespace(buffering_time=300, destination_address='localhost', destination_port=4444, frames_per_chunk=1024, frames_per_second=44100, input_device=None, list_devices=False, listening_port=4444, output_device=None, show_samples=False, show_stats=True)

Using device:

   0 HDA Intel PCH: ALC3246 Analog (hw:0,0), ALSA (2 in, 2 out)
   1 HDA Intel PCH: HDMI 0 (hw:0,3), ALSA (0 in, 8 out)
   2 HDA Intel PCH: HDMI 1 (hw:0,7), ALSA (0 in, 8 out)
   3 HDA Intel PCH: HDMI 2 (hw:0,8), ALSA (0 in, 8 out)
   4 HDA Intel PCH: HDMI 3 (hw:0,9), ALSA (0 in, 8 out)
   5 HDA Intel PCH: HDMI 4 (hw:0,10), ALSA (0 in, 8 out)
   6 sysdefault, ALSA (1

Much better!

### (Optional) Remove tc rules
They will be created after, again ...

In [9]:
!sudo tc qdisc del dev lo root
!tc qdisc show dev lo

qdisc noqueue 0: root refcnt 2 


## 3. Compression using [DEFLATE](https://en.wikipedia.org/wiki/DEFLATE)
We have "hidden" the jitter, but ... what happens if the available transmission bandwidth (the so called [network throughput](https://en.wikipedia.org/wiki/Throughput)) is not able to send and recive 1.41 Mbps?

A solution can be compression. The chunks can be compressed with [LZSS](https://en.wikipedia.org/wiki/Lempel-Ziv-Storer-Szymanski) (that is based on [LZ77](https://github.com/vicente-gonzalez-ruiz/LZ77)) and [Huffman Coding](https://en.wikipedia.org/wiki/Huffman_coding) (see also [this](https://vicente-gonzalez-ruiz.github.io/Huffman_coding/)).

We have choosen this text compression codec because:
1. It's fast.
2. Works well when repeated strings are found at the input.
3. I's available with [The Standard Python Library](https://docs.python.org/3/library/) ([zlib](https://docs.python.org/3/library/zlib.html)).

### Estimation of the throughput of the `localhost` link
Notice that this measurement depends heavely on the packet size, that using ping is limited to 64 KB. For this reason, the results can be only approximated :-/

In [10]:
!sudo tc qdisc del dev lo root
!tc qdisc show dev lo
!ping -c 1 -s 65527 localhost # Header length: 9 bytes

Error: Cannot delete qdisc with handle of zero.
qdisc noqueue 0: root refcnt 2 
ping: packet size 65527 is too large. Maximum is 65507


In [11]:
!printf "Gbps = "
!echo 65527*8/0.147/2/1000/1000 | bc -l

Gbps = 1.78304761904761904761


Too high! We need harder conditions for testing InterCom ...

### Let's limit the transmission bit-rate to 200 kbps
And also set the previous latency (200 ms) and jitter (200 ms).

In [27]:
!sudo tc qdisc add dev lo root handle 1: netem delay 100ms 100ms 25% distribution normal
!sudo tc qdisc add dev lo parent 1:1 handle 10: tbf rate 100kbit burst 128kbit limit 128kbit

In [ ]:
!ping -c 1 -s 65507 localhost

PING localhost (127.0.0.1) 65507(65535) bytes of data.


In [17]:
print("Throughput =", 65507*8/89/2, "kbps")

Throughput = 2944.1348314606744 kbps


### (Optional) Remove the previous rules

In [26]:
!sudo tc qdisc del dev lo parent 1:1 handle 10:
!sudo tc qdisc del dev lo root handle 1:
!tc qdisc show dev lo

qdisc noqueue 0: root refcnt 2 


### Let's try again

In [ ]:
!python ../buffer.py -b 300 --show_stats

Notice that some chunks are lost. DEFLATE may not be enought!

### Remove tc rules

In [ ]:
!sudo tc qdisc del dev lo parent 1:1 handle 10:
!sudo tc qdisc del dev lo root
!tc qdisc show dev lo

## 4. Bit-rate "control" through [quantization](https://github.com/vicente-gonzalez-ruiz/quantization/blob/master/digital_quantization.ipynb)

Quantization removes the less relevant information (mainly [electronic noise](https://en.wikipedia.org/wiki/Noise_(electronics)) ...) and helps to increase the [compression ratio](https://en.wikipedia.org/wiki/Compression_ratio). In lossy signal compression, [dead-zone quantizers](https://github.com/vicente-gonzalez-ruiz/quantization/blob/master/digital_quantization.ipynb) are commonly used.

In InterCom, we "control" the bit-rate in a lossely way because real-time bit-rate control through the quantization step $\Delta$ is computationally intensive (we must determine the [Rate/Distortion curve](https://en.wikipedia.org/wiki/Rate%E2%80%93distortion_theory) of the current chunk to find $\Delta$ before to quantize, compress and send it). The current implementation estimates the number of lost chunks per second and use this information to increase or decrease the quantization step for the chunks of the next second.

### Adapting to the current throughput

In [ ]:
!tc qdisc show dev lo
!sudo tc qdisc add dev lo root tbf rate 200kbit burst 1024kbit limit 1024kbit
!tc qdisc show dev lo
!python ../br_control.py -b 300 --show_stats

## 5. Spatial (inter-channel) decorrelation
The samples of a (stero) frame tend to have similar amplitudes. For this reason, we apply [Mid/Side coding](https://tecnologias-multimedia.github.io/study_guide/spatial_decorrelation/) (before quantization).

## 6. Temporal (intra-channel) decorrelation
The samples of a channel exhibit temporal redundancy. Therefore, we use a [Discrete Wavelet Transform (DWT)](https://tecnologias-multimedia.github.io/study_guide/temporal_decorrelation/) to exploit it (before quantization).

## 7. Work to do
1. Currently, quantization minimizes the MSE (Mean Square Error). Perceptual quantization can increase the compression ratio without increasing the perceived distortion.
2. Transmit video.